<a href="https://colab.research.google.com/github/Eeman1113/OCR_for_SBI/blob/main/Sbi_submisson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python
!pip install pytesseract
!pip install numpy
!pip install pyscreenshot

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
#OCR Part

# Import required packages
import cv2
import pytesseract

# Mention the installed location of Tesseract-OCR in your system
pytesseract.pytesseract.tesseract_cmd = (r'/opt/homebrew/Cellar/tesseract/5.1.0/bin/tesseract')

# Read image from which text needs to be extracted
img = cv2.imread(fn)

# Preprocessing the image starts

#negative of image 
img_not = cv2.bitwise_not(img)
img_not = cv2.bitwise_not(img_not)
cv2.imwrite("/content/img_not.png", img_not)
# Convert the image to gray scale
gray = cv2.cvtColor(img_not, cv2.COLOR_BGR2GRAY)
# Performing OTSU threshold
ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)

# Specify structure shape and kernel size.
# Kernel size increases or decreases the area
# of the rectangle to be detected.
# A smaller value like (10, 10) will detect
# each word instead of a sentence.
rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (18, 18))

# Applying dilation on the threshold image
dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1)

# Finding contours
contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL,
												cv2.CHAIN_APPROX_NONE)

# Creating a copy of image
im2 = img.copy()



# Looping through the identified contours
# Then rectangular part is cropped and passed on
# to pytesseract for extracting text from it
# Extracted text is then written into the text file
for cnt in contours:
  x, y, w, h = cv2.boundingRect(cnt)
	# Drawing a rectangle on copied image
  rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2)
	
	# Cropping the text block for giving input to OCR
  cropped = im2[y:y + h, x:x + w]
    
# Open the file in append mode
file = open("/content/recognized.txt", "a")
cv2.imwrite("/content/cropped.png", im2)
# Apply OCR on the cropped image
text = pytesseract.image_to_string(img_not, lang='eng')
l=[]
l.append(text)
time.sleep(0.6)

# Appending the text into file
file.write(text)
file.write("\n")

# Close the file
file.close
